In [2]:
from core import db
import analytiq
import pandas as pd
import datetime

In [16]:
df_listings = db.query("""
select
    date_trunc('month', d.date) as date,
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
    case when construction_year >= extract(year from d.date) - 5 and (sa.first_sale_date is null or sa.first_sale_date >= d.date) then 'new' else 'old' end as construction_category,
    c.name as city_name,
    avg((case when price > 10e6 and price < 300e6 and r.square_meters > 20 then price / r.square_meters else null end)) as sqm_price,
    count(distinct r.realestate_id) as listings,
    avg((d.date::date - lower(l.span)::date)::int) as listing_days
from core.listing l
join core.listing_price lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join core.realestate r
on lp.realestate_id = r.realestate_id
cross join generate_series(lower(lp.span)::date, (upper(lp.span)-make_interval(days:=1))::date, make_interval(days:=1)) as d(date)
left join (
  select realestate_code, min(registration_date::date) as first_sale_date
  from core.sale_agreement
  where is_current
  and is_deleted = false
  group by realestate_code
) sa
on r.realestate_code = sa.realestate_code
join core.realestate_type rt
on r.realestate_type_id = rt.realestate_type_id
join core.postal_code p
on r.postal_id = p.postal_id
join core.city c
on p.city_id = c.city_id
join core.region reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
group by 1,2,3,4
order by 1 desc
""")

In [ ]:
df_sales = db.query("""
with sales as (
    select * from core.sale_agreement where is_current and is_deleted = false
)
select
  date_trunc('month', s.agreement_date::date) as date,
  case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
  case when r.construction_year >= extract(year from s.registration_date::date) - 5 and (sa.first_sale_date is null or sa.first_sale_date < s.registration_date::date) then 'new' else 'old' end as construction_category,
  c.name as city_name,
  avg((case when purchase_price > 10e6 and purchase_price < 300e6 and r.square_meters > 20 then purchase_price / r.square_meters else null end)) as sqm_price,
  count(distinct r.realestate_id) as sales,
  avg(listing_days) as listing_days
from sales s
left join (
    select
        sa1.sale_agreement_id,
        sum(
          least(sa1.registration_date::date, upper(l1.span))::date -
          least(sa1.registration_date::date, lower(l1.span))::date
        ) as listing_days
    from core.listing l1
    join core.realestate r
    on l1.realestate_id = r.realestate_id
    join sales sa1
    on r.realestate_code = sa1.realestate_code
    and l1.span && daterange((registration_date::date - make_interval(months:=12))::date, registration_date::date, '[]')
    group by sa1.sale_agreement_id
) ld
on s.sale_agreement_id = ld.sale_agreement_id
join core.realestate r
on s.realestate_code = r.realestate_code
left join (
  select realestate_code, min(registration_date::date) as first_sale_date
  from sales
  group by realestate_code
) sa
on r.realestate_code = sa.realestate_code
join core.realestate_type rt
on r.realestate_type_id = rt.realestate_type_id
join core.postal_code p
on r.postal_id = p.postal_id
join core.city c
on p.city_id = c.city_id
join core.region reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
group by 1,2,3,4
""")

In [32]:
df_sales\
.loc[lambda r: r.construction_category == 'new']\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.pivot(index='date', columns='city_name', values='sqm_price')\
.analytiq.plot()

KeyError: 'date'

In [18]:
def make_plot(df):

    fig = analytiq.Figure('core')

    for i, column in enumerate(df.columns):

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i*2)
            ),
            showlegend=False
        )
    
    fig.set_title(
        "Meðal birtingatími <color 0>nýbygginga</color> og <color 2>eldri bygginga</color> fyrir sölu",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Meðalauglýsing á markaði áður en sala fer fram.</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()


    return fig



df_sales\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(listing_sales = lambda r: r.listing_days * r.sales)\
.assign(date=lambda r: r.date.dt.date)\
.groupby(['date', 'construction_category'], as_index=False)\
.agg(
    listing_sales = ('listing_sales', 'sum'),
    sales = ('sales', 'sum')
)\
.assign(listing_days = lambda r: r.listing_sales / r.sales)\
.pivot(index='date', columns='construction_category', values='listing_days')\
.loc[lambda r: r.index >= datetime.date(2018,1,1)]\
.loc[lambda r: r.index < datetime.date(2026,1,1)]\
.rolling(3).mean()\
.pipe(make_plot)\
.export("04-time-on-market-before-sale", scale=1.5)

In [19]:
df_sales_days_aggregate = df_sales\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(listing_sales = lambda r: r.listing_days * r.sales)\
.assign(date=lambda r: r.date.dt.date)\
.groupby(['date', 'construction_category'], as_index=False)\
.agg(
    listing_sales = ('listing_sales', 'sum'),
    sales = ('sales', 'sum')
)\
.assign(listing_days = lambda r: r.listing_sales / r.sales)\
.pivot(index='date', columns='construction_category', values='listing_days')\
.loc[lambda r: r.index >= datetime.date(2018,1,1)]\
.loc[lambda r: r.index < datetime.date(2026,1,1)]\
.rolling(3).mean()

In [20]:
df_listings_days_aggregate = df_listings\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(listing_sales = lambda r: r.listing_days * r.listings)\
.assign(date=lambda r: r.date.dt.date)\
.groupby(['date', 'construction_category'], as_index=False)\
.agg(
    listing_sales = ('listing_sales', 'sum'),
    sales = ('listings', 'sum')
)\
.assign(listing_days = lambda r: r.listing_sales / r.sales)\
.pivot(index='date', columns='construction_category', values='listing_days')\
.loc[lambda r: r.index >= datetime.date(2018,1,1)]\
.loc[lambda r: r.index < datetime.date(2026,1,1)]\
.rolling(3).mean()

In [21]:
fig = analytiq.Figure('core')

fig.as_subplot(
    subplot_titles=["Eldri byggingar", "Nýbyggingar"],
    rows=1, cols=2, 
    shared_yaxes=True
)

for i, col in enumerate(['old', 'new']):

    fig.add_scatter(
        x = df_sales_days_aggregate.index,
        y = df_sales_days_aggregate[col],
        line = dict(
            color = fig.colors(4)
        ),
        row=1, col=i+1,
        showlegend=False
    )
    fig.add_scatter(
        x = df_listings_days_aggregate.index,
        y = df_listings_days_aggregate[col],
        line = dict(
            color = fig.colors(5)
        ),
        row=1, col=i+1,
        showlegend=False
    )

for annotation in fig.layout.annotations:
    annotation['font'] = fig.get_font(size=20)
    annotation['text'] = analytiq.ph.bold(annotation['text'])

fig.set_title(
    "Meðaltals <color 5>birtingartími</color> og <color 4>sölutími</color> fasteigna",
)

fig.add_annotation(
    x = -0.027,
    y = 1.13,
    showarrow=False,
    font = fig.get_font(size=18),
    text = "<i>Meðalauglýsing á markaði áður en sala fer fram.</i>",
    xref='paper', yref='paper',
    xanchor='left',
    align='left'

)

fig.update_layout(
    margin_t = 150,
)

fig.add_logo()

fig.export("04-time-on-market-vs-time-before-sale", scale=1.5)

fig.show()

In [22]:
from analytiq.utils import join_split_columns

In [23]:
def make_plot(df):

    fig = analytiq.Figure('core')

    fig.add_scatter(
        x = df.index,
        y = df['sqm_price_s_Fjölbýli'],
        line = dict(
            color = fig.colors(3),
            width = 3
        ),
        showlegend=False
    )
    fig.add_scatter(
        x = df.index,
        y = df['sqm_price_l_Fjölbýli'],
        line = dict(
            color = fig.colors(5),
            width = 3
        ),
        showlegend=False
    )
    
    fig.set_title(
        "Fermetraverð <color 5>á söluskrá</color> og <color 3>seldra eigna</color>",
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Íbúðarhúsnæðis á höfuðborgarsvæðinu</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
    )

    fig.add_logo()

    return fig



df_sales\
.merge(df_listings, on=['date', 'realestate_category_name', 'construction_category', 'city_name'], suffixes=['_s', '_l'])\
.assign(
    sqm_price_s__sales = lambda r: r.sqm_price_s * r.sales,
    sqm_price_l__sales = lambda r: r.sqm_price_l * r.sales,
    listing_days_s__sales = lambda r: r.listing_days_s * r.sales,
    listing_days_l__sales = lambda r: r.listing_days_l * r.sales
)\
.groupby(['date', 'realestate_category_name'], as_index=False)\
.agg(
    sqm_price_s__sales = ('sqm_price_s__sales', 'sum'),
    sqm_price_l__sales = ('sqm_price_l__sales', 'sum'),
    listing_days_s__sales = ('listing_days_s__sales', 'sum'),
    listing_days_l__sales = ('listing_days_l__sales', 'sum'),
    sales = ('sales', 'sum')
)\
.assign(
    sqm_price_s = lambda r: r.sqm_price_s__sales / r.sales,
    sqm_price_l = lambda r: r.sqm_price_l__sales / r.sales,
    listing_days_s = lambda r: r.listing_days_s__sales / r.sales,
    listing_days_l = lambda r: r.listing_days_l__sales / r.sales,
)\
.loc[:, ['date', 'realestate_category_name',  'sqm_price_s', 'sqm_price_l', 'listing_days_s', 'listing_days_l']]\
.pivot(index='date', columns=['realestate_category_name'], values=['sqm_price_s', 'sqm_price_l'])\
.rolling(3).mean()\
.pipe(join_split_columns)\
.pipe(make_plot)\
.export("04-sales-vs-listing-price", scale=1.5)

In [24]:
df_listings = db.query("""
select
    d.date as date,
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
    c.name as city_name,
    avg((case when price > 10e6 and price < 300e6 and r.square_meters > 20 then price / r.square_meters else null end)) as sqm_price,
    count(distinct r.realestate_id) as listings,
    avg((d.date::date - lower(l.span)::date)::int) as listing_days
from core.listing l
join core.listing_price lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join core.realestate r
on lp.realestate_id = r.realestate_id
cross join generate_series(lower(lp.span)::date, (upper(lp.span)-make_interval(days:=1))::date, make_interval(days:=1)) as d(date)
left join (
  select realestate_code, min(registration_date::date) as first_sale_date
  from core.sale_agreement
  where is_current
  and is_deleted = false
  group by realestate_code
) sa
on r.realestate_code = sa.realestate_code
join core.realestate_type rt
on r.realestate_type_id = rt.realestate_type_id
join core.postal_code p
on r.postal_id = p.postal_id
join core.city c
on p.city_id = c.city_id
join core.region reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
group by 1,2,3
order by 1 desc
""")

In [25]:
def make_plot(df):

    fig = analytiq.Figure('core')

    columns = [
         "Seltjarnarnesbær", "Mosfellsbær", "Garðabær", "Kópavogsbær", "Hafnarfjarðarkaupstaður", "Reykjavíkurborg"
    ]
    

    fig.update_layout(
        legend = dict(
            x = 0.0,
            y = 1.1,
            orientation = 'h'
        )
    )
    
    fig.set_title(
        "Fjöldi íbúða á markaði eftir sveitarfélagi",
    )


    fig.update_layout(
        margin_t = 150,
    )

    for i, column in enumerate(columns):
        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(i)
            ),
            name = column,
            showlegend=True
        )

        fig.export(f"total-on-market--{i}")

    return fig


df_listings\
.pivot_table(index='date', columns='city_name', values='listings', aggfunc='sum')\
.rolling(7).mean()\
.pipe(make_plot)

(0, 2)